In [1]:
import pandas as pd

owid = pd.read_csv(r"../data/owid-covid-data.csv")
owid['date_str'] = [''.join(item.split('-')) for item in owid['date']]
owid['countrycode_date_str'] = [';'.join(item) for item in owid[['iso_code', 'date_str']].values]
assert owid.shape[0] == len(owid['countrycode_date_str'].unique())
owid = owid.set_index('countrycode_date_str')
owid.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,date_str
countrycode_date_str,,,,,,,,,,,,,,,,,,,,,
AFG;20200224,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,20200224
AFG;20200225,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,20200225
AFG;20200226,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,20200226
AFG;20200227,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,20200227
AFG;20200228,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,20200228


In [2]:
oxcgrt_latest = pd.read_csv('../data/OxCGRT_latest.csv', index_col='CountryName')
oxcgrt_latest['countrycode_date_str'] = [';'.join([str(ele) for ele in item]) for item in oxcgrt_latest[['CountryCode', 'Date']].values]
print(oxcgrt_latest.shape, len(oxcgrt_latest['countrycode_date_str'].unique()))

groupedby_date_countrycode = oxcgrt_latest.groupby(['countrycode_date_str']).agg({'Date':'count',
                                                                                  'RegionName':lambda x:list(set(x)),
                                                                                  'CountryCode':lambda x:list(set(x))[0]})
big_countries = set(groupedby_date_countrycode['CountryCode'][groupedby_date_countrycode['Date']>1].value_counts().index.values)

# combine except those big countries
print('Big countries with more than one regions are:', big_countries)
isBigCountry = lambda x:True if x in big_countries else False
oxcgrt_latest['isBig'] = [isBigCountry(code) for code in oxcgrt_latest['CountryCode']]
print(oxcgrt_latest['isBig'].value_counts())

smallcountries_oxcgrt = oxcgrt_latest[oxcgrt_latest['isBig']==False].copy()
bigcountries_oxcgrt = oxcgrt_latest[oxcgrt_latest['isBig']==False].copy()

assert smallcountries_oxcgrt.shape[0] == len(smallcountries_oxcgrt['countrycode_date_str'].unique())

owid['isBig'] = [isBigCountry(code) for code in owid['iso_code']]
smallcountries_owid = owid[owid['isBig']==False]
bigcountries_owid = owid[owid['isBig']]

/home/shuang/miniconda3/envs/hackathon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(135161, 51) 89466
Big countries with more than one regions are: {'CAN', 'GBR', 'USA', 'BRA'}
False    87542
True     47619
Name: isBig, dtype: int64


In [4]:
# combine those small countries
combined_smallcountries = pd.concat([smallcountries_owid, 
                                     smallcountries_oxcgrt.set_index('countrycode_date_str')], axis=1, join='inner')
print(combined_smallcountries.shape)

(70938, 112)
